In [63]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

llm = ChatGroq(model='llama3-groq-70b-8192-tool-use-preview')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000015D214A7710>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000015D2150CB50>, model_name='llama3-groq-70b-8192-tool-use-preview', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [64]:
from langchain_core.prompts import ChatPromptTemplate,HumanMessagePromptTemplate,SystemMessagePromptTemplate,PromptTemplate

In [65]:
prompt =  ChatPromptTemplate.from_template("""Answer the user question 
                             ### input : {input}
                             Answer : """)

In [67]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [68]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate

In [69]:
# Create the System and Human messages
system_message = SystemMessage(content="You are a helpful assistant")
human_message = HumanMessage(content="What's the weather like in Boston?")

# Create a list of messages
messages = [system_message, human_message]

# Invoke the LLM with the list of messages
response = llm.invoke(messages)

print(response)

content="I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 28, 'total_tokens': 60, 'completion_time': 0.077891684, 'prompt_time': 0.004212736, 'queue_time': 0.036270040999999996, 'total_time': 0.08210442}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'stop', 'logprobs': None} id='run-bbd2b39a-4a2e-42a0-8109-728daf460827-0' usage_metadata={'input_tokens': 28, 'output_tokens': 32, 'total_tokens': 60}


In [70]:
llm.invoke(messages)

AIMessage(content="I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 28, 'total_tokens': 60, 'completion_time': 0.077798634, 'prompt_time': 0.004220107, 'queue_time': 0.035902267, 'total_time': 0.082018741}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'stop', 'logprobs': None}, id='run-e0bc1646-cf31-45ca-a561-3f2a756871a3-0', usage_metadata={'input_tokens': 28, 'output_tokens': 32, 'total_tokens': 60})

In [81]:
from ollama import chat
from ollama import ChatResponse

def add_two_numbers(a: int, b: int) -> int:
    """
    Add two numbers

    Args:
        a (int): The first number
        b (int): The second number

    Returns:
        int: The sum of the two numbers
    """
    return int(a) + int(b)

def subtract_two_numbers(a: int, b: int) -> int:
    """
    Subtract two numbers
    """
    return a - b

# Tools can still be manually defined and passed into chat
add_two_numbers_tool = {
    'type': 'function',
    'function': {
        'name': 'add_two_numbers',
        'description': 'Add two numbers',
        'parameters': {
            'type': 'object',
            'required': ['a', 'b'],
            'properties': {
                'a': {'type': 'integer', 'description': 'The first number'},
                'b': {'type': 'integer', 'description': 'The second number'},
            },
        },
    },
}

subtract_two_numbers_tool = {
    'type': 'function',
    'function': {
        'name': 'subtract_two_numbers',
        'description': 'Subtract two numbers',
        'parameters': {
            'type': 'object',
            'required': ['a', 'b'],
            'properties': {
                'a': {'type': 'integer', 'description': 'The first number'},
                'b': {'type': 'integer', 'description': 'The second number'},
            },
        },
    },
}

def get_user_input(prompt: str) -> int:
    """
    Prompt the user for input and validate it as an integer.

    Args:
        prompt (str): The prompt to display to the user.

    Returns:
        int: The validated integer input from the user.
    """
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

def handle_tool_call(tool, available_functions):
    """
    Handle a tool call by ensuring the function is available and calling it.

    Args:
        tool: The tool call object.
        available_functions: A dictionary of available functions.
    """
    if function_to_call := available_functions.get(tool.function.name):
        print('Calling function:', tool.function.name)
        print('Arguments:', tool.function.arguments)

        # Ensure all required arguments are present and valid
        args = tool.function.arguments
        if not args or 'a' not in args or 'b' not in args:
            print("Missing arguments. Please provide the required values.")
            a = get_user_input("Enter the first number (a): ")
            b = get_user_input("Enter the second number (b): ")
            args = {'a': a, 'b': b}

        # Call the function with the arguments
        try:
            result = function_to_call(**args)
            print('Function output:', result)
        except Exception as e:
            print('Error:', e)
    else:
        print('Function', tool.function.name, 'not found')

prompt = 'What is three plus ?'
print('Prompt:', prompt)

available_functions = {
    'add_two_numbers': add_two_numbers,
    'subtract_two_numbers': subtract_two_numbers,
}

response: ChatResponse = chat(
    'llama3.2',
    messages=[{'role': 'user', 'content': prompt}],
    tools=[add_two_numbers_tool, subtract_two_numbers_tool],
)

if response.message.tool_calls:
    # There may be multiple tool calls in the response
    for tool in response.message.tool_calls:
        handle_tool_call(tool, available_functions)
else:
    print("No tool calls were made.")
    

Prompt: What is three plus ?
Calling function: subtract_two_numbers
Arguments: {'a': '3', 'b': '?'}
Error: unsupported operand type(s) for -: 'str' and 'str'


In [86]:
from ollama import chat
from ollama import ChatResponse

def add_two_numbers(a: int, b: int) -> int:
    """
    Add two numbers

    Args:
        a (int): The first number
        b (int): The second number

    Returns:
        int: The sum of the two numbers
    """
    return int(a) + int(b)

def subtract_two_numbers(a: int, b: int) -> int:
    """
    Subtract two numbers
    """
    return a - b

# Tools can still be manually defined and passed into chat
add_two_numbers_tool = {
    'type': 'function',
    'function': {
        'name': 'add_two_numbers',
        'description': 'Add two numbers',
        'parameters': {
            'type': 'object',
            'required': ['a', 'b'],
            'properties': {
                'a': {'type': 'integer', 'description': 'The first number'},
                'b': {'type': 'integer', 'description': 'The second number'},
            },
        },
    },
}

subtract_two_numbers_tool = {
    'type': 'function',
    'function': {
        'name': 'subtract_two_numbers',
        'description': 'Subtract two numbers',
        'parameters': {
            'type': 'object',
            'required': ['a', 'b'],
            'properties': {
                'a': {'type': 'integer', 'description': 'The first number'},
                'b': {'type': 'integer', 'description': 'The second number'},
            },
        },
    },
}

def get_user_input(prompt: str) -> int:
    """
    Prompt the user for input and validate it as an integer.

    Args:
        prompt (str): The prompt to display to the user.

    Returns:
        int: The validated integer input from the user.
    """
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

def handle_tool_call(tool, available_functions):
    """
    Handle a tool call by ensuring the function is available and calling it.

    Args:
        tool: The tool call object.
        available_functions: A dictionary of available functions.
    """
    if function_to_call := available_functions.get(tool.function.name):
        print('Calling function:', tool.function.name)
        print('Arguments:', tool.function.arguments)

        # Ensure all required arguments are present and valid
        args = tool.function.arguments
        if not args or 'a' not in args or 'b' not in args:
            print("Missing arguments. Please provide the required values.")
            a = get_user_input("Enter the first number (a): ")
            b = get_user_input("Enter the second number (b): ")
            args = {'a': a, 'b': b}
        else:
            # Validate and convert arguments to integers
            try:
                args['a'] = int(args['a'])
                args['b'] = int(args['b'])
            except (ValueError, TypeError):
                print("Invalid arguments. Please provide valid integers.")
                a = get_user_input("Enter the first number (a): ")
                b = get_user_input("Enter the second number (b): ")
                args = {'a': a, 'b': b}

        # Call the function with the arguments
        try:
            result = function_to_call(**args)
            print('Function output:', result)
        except Exception as e:
            print('Error:', e)
    else:
        print('Function', tool.function.name, 'not found')

prompt = 'What is three plus ?'
print('Prompt:', prompt)

available_functions = {
    'add_two_numbers': add_two_numbers,
    'subtract_two_numbers': subtract_two_numbers,
}

response: ChatResponse = chat(
    'llama3.2',
    messages=[{'role': 'user', 'content': prompt}],
    tools=[add_two_numbers_tool, subtract_two_numbers_tool],
)

if response.message.tool_calls:
    # There may be multiple tool calls in the response
    for tool in response.message.tool_calls:
        handle_tool_call(tool, available_functions)
else:
    print("No tool calls were made.")

Prompt: What is three plus ?
Calling function: subtract_two_numbers
Arguments: {'a': '3', 'b': '?'}
Invalid arguments. Please provide valid integers.
Function output: -1


In [89]:
from langchain_groq.chat_models import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage

# Initialize the ChatGroq LLM
llm = ChatGroq(model='llama3-groq-70b-8192-tool-use-preview')

def add_two_numbers(a: int, b: int) -> int:
    """
    Add two numbers

    Args:
        a (int): The first number
        b (int): The second number

    Returns:
        int: The sum of the two numbers
    """
    return a + b

def subtract_two_numbers(a: int, b: int) -> int:
    """
    Subtract two numbers

    Args:
        a (int): The first number
        b (int): The second number

    Returns:
        int: The result of subtracting b from a
    """
    return a - b

# Tools definition
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'add_two_numbers',
            'description': 'Add two numbers',
            'parameters': {
                'type': 'object',
                'required': ['a', 'b'],
                'properties': {
                    'a': {'type': 'integer', 'description': 'The first number'},
                    'b': {'type': 'integer', 'description': 'The second number'},
                },
            },
        },
    },
    {
        'type': 'function',
        'function': {
            'name': 'subtract_two_numbers',
            'description': 'Subtract two numbers',
            'parameters': {
                'type': 'object',
                'required': ['a', 'b'],
                'properties': {
                    'a': {'type': 'integer', 'description': 'The first number'},
                    'b': {'type': 'integer', 'description': 'The second number'},
                },
            },
        },
    },
]

# Available functions dictionary
available_functions = {
    'add_two_numbers': add_two_numbers,
    'subtract_two_numbers': subtract_two_numbers,
}

system_message = SystemMessage(content="You are a helpful assistant. Use the provided tools to perform calculations.")
human_message = HumanMessage(content="Use the `add_two_numbers or subtract_two_numbers` function to calculate the subraction of tweleve and 2.")

# Create a list of messages
messages = [system_message, human_message]

# Invoke the LLM with the list of messages
response = llm.invoke(input=messages, tools=tools)

print(response)

# Handle tool calls if present
if hasattr(response, 'tool_calls') and response.tool_calls:
    for tool_call in response.tool_calls:
        # Print the tool_call object to inspect its structure
        print("Tool Call Object:", tool_call)

        # Extract function name and arguments based on the actual structure
        if hasattr(tool_call, 'function'):
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments
        elif isinstance(tool_call, dict):
            function_name = tool_call.get('name')
            arguments = tool_call.get('args')  # Use 'args' instead of 'arguments'
        else:
            print("Unexpected tool_call structure:", tool_call)
            continue

        # Check if the function exists
        if function_name in available_functions:
            function_to_call = available_functions[function_name]
            print('Calling function:', function_name)
            print('Arguments:', arguments)

            # Handle missing or invalid arguments
            if arguments is None:
                print("Arguments are missing or invalid.")
                continue

            # Call the function with the parsed arguments
            try:
                output = function_to_call(**arguments)
                print('Function output:', output)
            except TypeError as e:
                print("Error calling function:", e)
        else:
            print('Function', function_name, 'not found')
else:
    print('Response:', response.content)

content='' additional_kwargs={'tool_calls': [{'id': 'call_drh2', 'function': {'arguments': '{"a": 12, "b": 2}', 'name': 'subtract_two_numbers'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 357, 'total_tokens': 390, 'completion_time': 0.078450926, 'prompt_time': 0.02759074, 'queue_time': 0.037947749, 'total_time': 0.106041666}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-cfc9e156-a5ac-4d68-9bd8-5cb216cc85fd-0' tool_calls=[{'name': 'subtract_two_numbers', 'args': {'a': 12, 'b': 2}, 'id': 'call_drh2', 'type': 'tool_call'}] usage_metadata={'input_tokens': 357, 'output_tokens': 33, 'total_tokens': 390}
Tool Call Object: {'name': 'subtract_two_numbers', 'args': {'a': 12, 'b': 2}, 'id': 'call_drh2', 'type': 'tool_call'}
Calling function: subtract_two_numbers
Arguments: {'a': 12, 'b': 2}
Function output: 10
